
# ViEWS 3: presenting monthly results


Fatalities002 version

This notebook presents a standardized set of visualizations based on the monthly forecasts produced by the cm_futurepredictions notebook, for use in newsletters, webpage, etc.

#### Ensure the correct version of the mapper is installed. Version 1.4.0 as of 8/24/2022

Run "pip install views_mapper2 --upgrade" if out of date




In [ ]:
!pip freeze | grep views-mapper2

# Defining universals and obtain the data

In [ ]:
# Basics
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.cbook as cbook

import warnings
warnings.filterwarnings('ignore')

# Views 3
from viewser.operations import fetch
from viewser import Queryset, Column
import views_runs
from views_partitioning import data_partitioner, legacy
from stepshift import views
import views_dataviz
from views_runs import storage, ModelMetadata
from views_runs.storage import store, retrieve, fetch_metadata
from views_forecasts.extensions import *

# Packages from this repository, Tools folder
import sys
sys.path.append('../')
sys.path.append('../Tools')
sys.path.append('../Intermediates')
sys.path.append('../SystemUpdates')
from Ensembling import CalibratePredictions, RetrieveStoredPredictions, mean_sd_calibrated, gam_calibrated
from FetchData import FetchData, RetrieveFromList
from ViewsEstimators import *

In [ ]:
# Common parameters:
dev_id = 'Fatalities002'
run_id = dev_id 
EndOfHistory = 509
#prod_id = '2022_04_t01'
level = 'cm'

#note the home path has been automated with the os.path.expanduser function
#no need to adjust the folder as long as ViEWS on the dropbox is the destination

import os
home = os.path.expanduser("~")
Mydropbox = home + '/Dropbox (ViEWS)/ViEWS/'
Monthly_updates = Mydropbox + 'DataReleases/MonthlyUpdates/'



# Plan for this notebook

The notebook will produce a standardized set of visualizations for the monthly updates and store in the dropbox directory structure.

Rough loop structure:

for [cm, pgm]:

    for [sb]: # -- os and ns will come later
    
        Prediction maps for ensemble, possibly in two variants: predicted number of fatalities for s = [3,6,12,36]
        
        Change maps: change in predictions issued now and those issued **one** and three months earlier, for s = [3,12]
        
        Change maps: change in predictions issued now relative to last observed fatality count, for s = [3,12]
        
        Bar charts of **total** fatalities **per ame country** for next 12/36 months
        
        Line graphs: fatalities per month over time globally and for some selected countries and regions
        
    

## Prediction maps

- Prediction maps for ensemble, standard + chosen tiles: predicted number of fatalities for s = [3,6,12,36]
    - s=3 predictions printed as a table for all of ame
    
- Line graphs of cm predictions per month, 36 months, all countries in AME
- One line graph of cm predictions per month, 36 months, for countries with top 5 highest fatalities (in total over this period?)

- Pie chart showing the proportion of predicted fatalities per country – amongst countries for which we predict more than 50 fatalities over the next 3 months 
- Pie chart showing the proportion of predicted fatalities per country – amongst countries for we predict more than 100 fatalities over the next 3 months 

- Bar charts of total number of fatalities per ame country over the next 12 and 36 months (separate files)

## Change in prediction maps

- Change in predictions issued now and those issued **one** and three months earlier, for s = [3,12]
    - In number of fatalities
    - Categorical to minimise information overflow: e.g., 0-5% change, 5-10% change, 10-20% change, 20-50% change, more than 50% change? (up or down)
    - Both options above printed as a table for top 5 increases and top 5 decreases since last month at s=3
- Change in predictions issued now relative to last observed fatality count, for s = [3,12]
    - In number of fatalities
    - Categorical to minimise information overflow: e.g., +/- 0-5% change, 5-10% change, 10-20% change, 20-50% change, more than 50% change? (Suggestions?) 
    - Both options above printed as a table for top 5 increases and top 5 decreases since last month at s=3

## Surrogate maps 

- Surrogate model prediction maps (see cm_futurepredictions notebook)
- Surrogate model change maps: change in prediction for s=3 between now and those issued last month (see cm_futurepredictions notebook)

## Feature maps

- See ALM's feature notebook for inspo (ALM can also insert these). 
    - GED data for EndOfHistory at cm and pgm (without transforms to fill in for missing data), with option to choose cmap and scale on the color bar (viridis and single log scale as default, with number of fatalities as labels)
    - Change (number of fatalities and %) in GED data for EndofHistory relative to EndofHistory-1, cm and pgm